In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Read the CSV and Perform Basic Data Cleaning

In [2]:
# Load the data
file = 'FoodAccess.csv'
df = pd.read_csv(file)
# check data type
df.dtypes 

CensusTract             int64
state_county           object
Urban                   int64
Pop2010                 int64
OHU2010                 int64
PovertyRate           float64
MedianFamilyIncome    float64
TractLOWI             float64
LA1and10                int64
dtype: object

In [3]:
# Find null values
for column in df.columns :
    print(f"Column {column} has {df[column].isnull().sum()} null values")

Column CensusTract has 0 null values
Column state_county has 0 null values
Column Urban has 0 null values
Column Pop2010 has 0 null values
Column OHU2010 has 0 null values
Column PovertyRate has 3 null values
Column MedianFamilyIncome has 748 null values
Column TractLOWI has 4 null values
Column LA1and10 has 0 null values


In [4]:
# Remove null rows
#--Median Family Income has 748 null values which is 1.04%. This figure is insignificant compared to the importance of the
#feature towards prediction. So we deciide to drop null values than remove the column.
df=df.dropna()

In [7]:
#Find duplicate entries
df = df.drop_duplicates()
df['PctLowI']=(df.TractLOWI/df.Pop2010)*100
df = df[['CensusTract', 'state_county','Urban','Pop2010', 'OHU2010','PovertyRate', 'MedianFamilyIncome','PctLowI','LA1and10']]
df.head()

,CensusTract,state_county,Urban,Pop2010,OHU2010,PovertyRate,MedianFamilyIncome,PctLowI,LA1and10
0,1001020100,Alabama_Autauga County,1,1912,693,11.3,81250.0,23.797071,1
1,1001020200,Alabama_Autauga County,1,2170,743,17.9,49000.0,36.958525,1
2,1001020300,Alabama_Autauga County,1,3373,1256,15.0,62609.0,38.719241,1
3,1001020400,Alabama_Autauga County,1,4386,1722,2.8,70607.0,21.021432,1
4,1001020500,Alabama_Autauga County,1,10766,4082,15.2,96334.0,20.824819,1


# Define Model

In [8]:
# Create features
X = df.drop(columns=['CensusTract','state_county',"LA1and10"]) 

# Create target
y = df["LA1and10"]

# Split model
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


### Random Forest

In [9]:
# Create Random Forest Classifier model
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [10]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
# Making predictions using the testing data.
rf_y_pred = rf_model.predict(X_test_scaled)
# Evaluate the model
print(f"Random Forest model accuracy: {accuracy_score(y_test,rf_y_pred):.3f}")

Random Forest model accuracy: 0.689


In [11]:
confusion_matrix(y_test,rf_y_pred)

array([[9212, 1935],
       [3649, 3150]])

In [12]:
rf_model.feature_importances_

array([0.06537311, 0.20518136, 0.19471008, 0.1659295 , 0.18270574,
       0.18610022])

In [13]:
importances= pd.DataFrame(data={'Importance': rf_model.feature_importances_}, index=X.columns)
importances

,Importance
Urban,0.065373
Pop2010,0.205181
OHU2010,0.194710
PovertyRate,0.165930
MedianFamilyIncome,0.182706
PctLowI,0.186100


In [28]:
from sklearn.ensemble import GradientBoostingClassifier

# Create a classifier object
learning_rates = [0.005, 0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=100,
                                            learning_rate=learning_rate,
                                            max_features=6,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test)))
    print()

Learning rate:  0.005
Accuracy score (training): 0.657
Accuracy score (validation): 0.654

Learning rate:  0.05
Accuracy score (training): 0.698
Accuracy score (validation): 0.695

Learning rate:  0.1
Accuracy score (training): 0.703
Accuracy score (validation): 0.695

Learning rate:  0.25
Accuracy score (training): 0.710
Accuracy score (validation): 0.699

Learning rate:  0.5
Accuracy score (training): 0.718
Accuracy score (validation): 0.698

Learning rate:  0.75
Accuracy score (training): 0.720
Accuracy score (validation): 0.693

Learning rate:  1
Accuracy score (training): 0.724
Accuracy score (validation): 0.689



In [29]:
# Choose a learning rate and create classifier
classifier = GradientBoostingClassifier(n_estimators=100,
                                        learning_rate=0.25,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train)

# Make Prediction
predictions = classifier.predict(X_test_scaled)

In [30]:
classifier.feature_importances_
boosted_importances= pd.DataFrame(data={'Importance':classifier.feature_importances_}, index=X.columns)
boosted_importances

,Importance
Urban,0.292881
Pop2010,0.216148
OHU2010,0.165232
PovertyRate,0.182464
MedianFamilyIncome,0.084333
PctLowI,0.058941


In [31]:
importances

,Importance
Urban,0.065373
Pop2010,0.205181
OHU2010,0.194710
PovertyRate,0.165930
MedianFamilyIncome,0.182706
PctLowI,0.186100


In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.6972027192689179


In [17]:
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       0.72      0.84      0.78     11147
           1       0.64      0.46      0.53      6799

    accuracy                           0.70     17946
   macro avg       0.68      0.65      0.66     17946
weighted avg       0.69      0.70      0.68     17946



In [20]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,9391,1756
Actual 1,3678,3121
